In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.models as models

In [2]:
import random

In [3]:
import numpy as np

In [4]:
from PIL import Image

In [5]:
import os
import cv2

In [6]:
im_art= cv2.imread('./art.jpg')

In [7]:
im_true=cv2.imread('./test_pic.jpeg')

In [8]:
im_art=cv2.resize(im_art,(400,400),interpolation=cv2.INTER_NEAREST)

In [9]:
im_true=cv2.resize(im_true,(400,400),interpolation=cv2.INTER_NEAREST)

In [10]:
im_art=torch.tensor(im_art).float()

In [11]:
im_true=torch.tensor(im_true).float()

In [12]:
from torchvision.models import vgg19

In [13]:
vgg19=vgg19(pretrained=True).features.eval()

In [14]:
for i in range(len(vgg19)):
    print(vgg19[i])


Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True

In [15]:
im_art=im_art.permute(2,0,1)

In [16]:
im_true=im_true.permute(2,0,1)

In [17]:
im_true.shape

torch.Size([3, 400, 400])

In [18]:
im_art=im_art.resize(1,3,400,400)
im_true=im_true.resize(1,3,400,400)

C:\Users\zhou\anaconda3\lib\site-packages\torch\tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [19]:
['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']

['block1_conv2',
 'block2_conv2',
 'block3_conv3',
 'block4_conv3',
 'block5_conv3']

In [20]:
def content_loss(content,combination):
    return ((content-combination)**2).sum()

In [21]:
def choose_content(im_true):
    for i in range(len(vgg19)):
        im_true=vgg19[i](im_true)
        if i==7:
            final_true=im_true
            break
    return final_true
        

In [22]:
im_true=choose_content(im_true)

In [23]:
def choose_style(im_art):
    list_style=[]
    for i in range(len(vgg19)):
        im_art=vgg19[i](im_art)
        if i in [2,7,14,23,32]:
            list_style.append(im_art)
    return list_style

In [24]:
list_style=choose_style(im_art)

In [25]:
list_style[0][0][0].shape

torch.Size([400, 400])

In [26]:
def gram(list_style):
    list_gram=[]
    for i in range(len(list_style)):
        c,h,w=list_style[i][0].shape
        style_tensor=list_style[i][0].reshape(c,(h*w))
        gram=style_tensor.mm(style_tensor.T)
        list_gram.append(gram)
    return list_gram
        
    

In [27]:
list_style_gram=gram(list_style)

In [28]:
def style_loss(list_style_gram,list_style_gram_combination):
    sum_style_loss=0
    for i in range(len(list_style_gram)):
        M,N=list_style_gram[i].shape
        sum_style_loss+=(((list_style_gram[i]-list_style_gram_combination[i])**2).sum())/(4*(M**2)*(N**2)*len(list_style_gram))
    return sum_style_loss

In [29]:
im_true1=cv2.imread('./test_pic.jpeg')

In [30]:
im_true1.shape

(816, 1228, 3)

In [31]:
im_true1=cv2.resize(im_true1,(400,400),interpolation=cv2.INTER_NEAREST)

In [32]:
im_true1=torch.tensor(im_true1).float()

In [33]:
im_true1=im_true1.permute(2,0,1)

In [34]:
im_true1=im_true1.resize(1,3,400,400)

In [35]:
def get_combination(im_true,list_style_gram,im_true1,learnrate,maxiter):
    Iter=0
    im_true1=Variable(im_true1,requires_grad=True)
    for i in range(maxiter):
        im_content_loss=content_loss(im_true,choose_content(im_true1))
        im_style_loss=style_loss(list_style_gram,gram(choose_style(im_true1)))
        all_loss=0.025*im_content_loss+5*im_style_loss
        all_loss.backward(retain_graph=True)
        im_true1.data=im_true1.data-learnrate*im_true1.grad.data
        im_true1.grad.data.zero_()
    return im_true1

In [36]:
get_it=get_combination(im_true,list_style_gram,im_true1,0.000003,50)

In [37]:
a=get_it.data.resize(3,400,400)

In [38]:
a=a.permute(1,2,0)

In [39]:
a=a.numpy()

In [40]:
cv2.imshow('a',a)
cv2.waitKey(0)
cv2.imwrite('C:\\Users\\zhou\\Desktop\\aaa.jpg',a)

True

In [ ]:
im_true1=cv2.imread('./test_pic.jpeg')


In [ ]:
im_true1=cv2.resize(im_true1,(400,400),interpolation=cv2.INTER_NEAREST)

In [ ]:
im_true1=torch.tensor(im_true1)

In [ ]:
im_true1.shape

In [ ]:
im_true1=im_true1.permute(2,0,1)
im_true1=im_true1.resize(1,3,400,400)
im_true1=im_true1.resize(3,400,400)
im_true1=im_true1.permute(1,2,0)
im_true1=im_true1.numpy()

In [ ]:
cv2.imshow('aa',im_true1)
cv2.waitKey(0)